In [ ]:
# Parameters. Set defaults here.
# Times Square replaces this cell with the user's parameters.
record_limit = '999'

<a class="anchor" id="imports"></a>
## Imports and General Setup

In [ ]:
# Only use packages available in the Rubin Science Platform
import requests
from collections import defaultdict
import pandas as pd
from pprint import pp, pformat
from urllib.parse import urlencode
from IPython.display import FileLink, display_markdown
from matplotlib import pyplot as plt
import os
from astropy.time import Time, TimeDelta

In [ ]:
# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get('EXTERNAL_INSTANCE_URL'):
    print('Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch....')
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype >/dev/null
import lsst.ts.logging_and_reporting.efd as lre

In [ ]:
summit = 'https://summit-lsp.lsst.codes'
usdf = 'https://usdf-rsp-dev.slac.stanford.edu'
tucson = 'https://tucson-teststand.lsst.codes'

# RUNNING_INSIDE_JUPYTERLAB is True when running under Times Square
server = os.environ.get('EXTERNAL_INSTANCE_URL', summit)


In [ ]:
sad = lre.EfdAdapter(server_url=usdf)

In [ ]:
# Find topics that have data in the past DAYS ending today.
populated, errors, count = await sad.find_populated_topics(days=1)
display(errors)
print(f'Successfully queried {count} topics.  Failed query against {len(errors)} topics.')

print(f'Topics that failed efd_client.select_time_series are: ')
print('\n'.join(list(errors.keys())))

display({topic: len(fields) for topic,fields in populated.items()})

In [ ]:
display(sad.min_date, sad.max_date)
ww = await sad.get_weather(days=1)

In [ ]:
print('\n'.join(sad.client.query_history))

In [ ]:
ww

In [ ]:
import random
all_topics = await sad.client.get_topics()
print(f'Found {len(all_topics)} total topics.')
topics = [top for top in all_topics if 'logevent_' in top and 'lsst.sal.' in top]
print(f'Found {len(topics)} topics containing "logevent_" and "lsst.sal.".')
sample_count = 13
print(f'A random sample of {sample_count} is:')
display(random.sample(topics,sample_count))

In [ ]:
random.sample(all_topics,30)

In [ ]:
fields = [f for f in await sad.client.get_fields('lsst.sal.ATAOS.logevent_logMessage') if 'private_' not in f]
fields

In [ ]:
t = Time([], format='isot', scale='utc')
days = TimeDelta(2, format='jd')
query = sad.client.build_time_range_query('lsst.sal.ATAOS.logevent_logMessage', fields, 
                                          start=t.now()-days, end=t.now())
query

In [ ]:
days = TimeDelta(2, format='jd')
print(await sad.client.get_fields('lsst.sal.MTM2.command_positionMirror') )
positions = await sad.client.select_time_series('lsst.sal.MTM2.command_positionMirror', 
                                    ['x', 'xRot', 'y', 'yRot','z','zRot'], 
                                    start=t.now(), end=days*2, is_window=True, index=301)
positions[:2]

In [ ]:
from astropy.time import Time, TimeDelta
times = ['2020-01-01T00:00:00.123456789', '2021-01-01T00:00:00']
t = Time(times, format='isot', scale='utc')
days = TimeDelta(2)
query = sad.client.build_time_range_query('lsst.sal.ATAOS.logevent_logMessage', fields, 
                                          start=t.now()-days, end=t.now())
query

In [ ]:
t_now = t.now()
t_old = t_now - 5
t_old, t_now

In [ ]:
await sad.client.select_time_series('lsst.sal.ATAOS.logevent_logMessage', ['message','priority'], 
                                    start=t.now(), end=days*2, is_window=True, index=301)

In [ ]:
print(f'{days=} {t.now()=}')
async def query_night(topic, fields, index=None):
    """Query an EFD topic and field(s) from sunset to sunrise."""
    # note that sunset/sunrise_time as set from global state
    # outside this function
    return await sad.client.select_time_series(
        topic,
        fields,
        start=t.now(),
        end=days, 
        index=index,
        is_window=True,  # time range is centered on START. Width of range given by END.
    )

In [ ]:
await sad.client.get_fields('lsst.sal.ESS.airFlow') # BUG, returns duplicate field names

In [ ]:
ess_wind_df = await query_night("lsst.sal.ESS.airFlow", ["speed", "direction"], index=301) # m/s

In [ ]:
len(ess_wind_df)

In [ ]:
ess_wind_df[:10]

In [ ]:
schema = await sad.client.get_schema("lsst.sal.ATCamera.vacuum")
schema


In [ ]:
dir(sad.client)

<a class="anchor" id="setup_source"></a>
## Setup Source

In [ ]:
type(sad.client)

In [ ]:
sad.client.build_time_range_query?

In [ ]:
sad.client.get_schema_topics?

In [ ]:
sad.client.get_schema?

In [ ]:
schema = await sad.client.get_schema('lsst.sal.ATCamera.command_start')

In [ ]:
import lsst_efd_client.efd_helper
lsst_efd_client.efd_helper??

In [ ]:
md = f'### Will retrieve from {service}'
display_markdown(md, raw=True)

In [ ]:
recs = None
ok = True

# is_human=either&is_valid=either&offset=0&limit=50' 
# site_ids=tucson&message_text=wubba&min_level=0&max_level=999&user_ids=spothier&user_agents=LOVE
# tags=love&exclude_tags=ignore_message
qparams = dict(is_human='either',
               is_valid='either',
               limit=limit,
              )
qstr = urlencode(qparams)
url = f'{service}/messages?{qstr}'

ignore_fields = set(['tags', 'urls', 'message_text', 'id', 'date_added', 
                     'obs_id', 'day_obs', 'seq_num', 'parent_id', 'user_id',
                     'date_invalidated', 'date_begin', 'date_end',
                     'time_lost', # float
                     #'systems','subsystems','cscs',  # values are lists, special handling
                    ])

In [ ]:
display_markdown(f'## Get (up to {limit}) Records', raw=True)

In [ ]:
# TODO Often fails on first request.  Find out why!
try:
    response = requests.get(url, timeout=timeout)
except:
    pass 
    
try:
    print(f'Attempt to get logs from {url=}')
    response = requests.get(url, timeout=timeout)
    response.raise_for_status()
    recs = response.json()
    flds = set(recs[0].keys())
    facflds = flds - ignore_fields
    # facets(field) = set(value-1, value-2, ...)
    facets = {fld: set([str(r[fld])
                for r in recs if not isinstance(r[fld], list)]) 
                    for fld in facflds}
except Exception as err:
    ok = False
    print(f'ERROR getting {log} from {env=} using {url=}: {err=}')
numf = len(flds) if ok else 0
numr = len(recs) if ok else 0
print(f'Retrieved {numr} records, each with {numf} fields.')

<a class="anchor" id="table"></a>
## Tables of (mostly raw) results

### Fields names provided in records from log.

In [ ]:
pd.DataFrame(flds, columns=['Field Name'])

### Facets from log records.
A *facet* is the set all of values found for a field in the retrieved records. Facets are only calculated for some fields.

In [ ]:
display(pd.DataFrame.from_dict(facets, orient='index'))
display(facets)

### Table of selected log record fields.
Table can be retrieved as CSV file for local use.

In [ ]:
cols = ['date_added', 'time_lost']
df = pd.DataFrame(recs)[cols]

# Allow download of CSV version of DataFrame
csvfile = 'tl.csv'
df.to_csv(csvfile)
myfile = FileLink(csvfile)
print('Table available as CSV file: ')
display(myfile)
df

In [ ]:
df = pd.DataFrame(recs)
df

<a class="anchor" id="plot"></a>
## Plots from log

In [ ]:
x = [r['date_added'] for r in recs]
y = [r['time_lost'] for r in recs]
plt.plot(x, y) 
plt.show()

<a class="anchor" id="raw_analysis"></a>
## Raw Content Analysis

### Example of one record

In [ ]:
rec = recs[-1]

msg = rec["message_text"]
md = f'Message text from log:\n> {msg}'
display_markdown(md, raw=True)

display(rec)

<a class="anchor" id="elicitation"></a>
## Stakeholder Elicitation

In [ ]:
#EXTERNAL_INSTANCE_URL
ed = dict(os.environ.items())
with pd.option_context('display.max_rows', None,):
    print(pd.DataFrame(ed.values(), index=ed.keys()))